In [672]:
#!pip install rake_nltk
#!pip install --upgrade pip

In [673]:
import nltk
import pandas as pd
import re
import unicodedata
import numpy as np

from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer #tokenizes a collection of words extracted from a text doc
from ast import literal_eval #This evaluates whether an expresion is a Python datatype or not


In [674]:
cosmetics_path = "./data/cosmetics.csv"
raw_df = pd.read_csv(cosmetics_path)
raw_df.head()

,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolatum, Glycerin, Isohexadecane, Microcrystalline Wax, Lanolin Alcohol, Citrus Aurantifolia (Lime) Extract, Sesamum Indicum (Sesame) Seed Oil, Eucalyptus Globulus (Eucalyptus) Leaf Oil, Sesamum Indicum (Sesame) Seed Powder, Medicago Sativa (Alfalfa) Seed Powder, Helianthus Annuus (Sunflower) Seedcake, Prunus Amygdalus Dulcis (Sweet Almond) Seed Meal, Sodium Gluconate, Copper Gluconate, Calcium Gluconate, Magnesium Gluconate, Zinc Gluconate, Magnesium Sulfate, Paraffin, Tocopheryl Succinate, Niacin, Water, Beta-Carotene, Decyl Oleate, Aluminum Distearate, Octyldodecanol, Citric Acid, Cyanocobalamin, Magnesium Stearate, Panthenol, Limonene, Geraniol, Linalool, Hydroxycitronellal, Citronellol, Benzyl Salicylate, Citral, Sodium Benzoate, Alcohol Denat., Fragrance.",1,1,1,1,1
1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butylene Glycol, Pentylene Glycol, Water, Sodium Benzoate, Methylparaben, Sorbic Acid.",1,1,1,1,1
2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Cetearyl Alcohol, Cetearyl Olivate, Sorbitan Olivate, Sclerocarya Birrea Seed Oil, Bacillus/Soybean/ Folic Acid Ferment Extract, Nymphaea Alba Root Extract, sh-Oligopeptide-1, sh-Oligopeptide-2, sh-Polypeptide-1, sh-Polypeptide-9, sh-Polypeptide-11, Copper Palmitoyl Heptapeptide-14, Heptapeptide-15 Palmitate, Palmitoyl Tetrapeptide-7, Palmitoyl Tripeptide-1, Alanine, Arginine, Glycine, Histidine, Isoleucine, Phenylalanine, Proline, Serine, Threonine, Valine, Acetyl Glutamine, Coconut Alkanes , Coco-Caprylate/Caprate, Sodium Hyaluronate, Aspartic Acid, Linoleic Acid, Linolenic Acid, Lecithin, Butylene Glycol, Polyvinyl Alcohol, Sodium Lactate, Sodium PCA, PCA, Sorbitan Isostearate, Carbomer, Polysorbate 20, Polysorbate 60, Lactic Acid/Glycolic Acid Copolymer, Hydroxyethyl Acrylate/Sodium Acryloyldimethyl Taurate Copolymer, Xanthan Gum, Isomalt, 1,2-Hexanediol, Caprylyl Glycol, Chlorphenesin, Phenoxyethanol, Tocopherol, Sodium Benzoate, Phenylpropanol, Glyceryl Caprylate, Symphytum Officinale Callus Culture Extract.",1,1,1,1,0
3,Moisturizer,LA MER,The Moisturizing Soft Cream,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, Petrolatum, Glyceryl Distearate, Phenyl Trimethicone, Butylene Glycol, Hydrogenated Vegetable Oil, Cholesterol, Butyrospermum Parkii (Shea Butter), Steareth-10, Dimethicone, Glyceryl Stearate Se, Polysilicone-11, Sesamum Indicum (Sesame) Seed Oil, Medicago Sativa (Alfalfa) Seed Powder, Helianthus Annuus (Sunflower) Seedcake, Prunus Amygdalus Dulcis (Sweet Almond) Seed Meal, Eucalyptus Globulus (Eucalyptus) Leaf Oil, Sodium Gluconate, Copper Gluconate, Calcium Gluconate, Magnesium Gluconate, Zinc Gluconate, Tocopheryl Succinate, Niacin, Sesamum Indicum (Sesame) Seed Powder, Water, Citrus Aurantifolia (Lime) Peel Extract, Laminaria Digitata Extract, Crithmum Maritimum Extract, Salicornia Herbacea Extract, Plankton Extract, Chlorella Vulgaris Extract, Glycine Soja (Soybean) Seed Extract, Glycerin, Caffeine, Sea Salt/Maris Sal/Sel Marin, Micrococcus Lysate, Diethylhexyl Succinate, Adenosine Phosphate, Creatine, Hydrolyzed Algin, Isocetyl Stearoyl Stearate, Cetyl Alcohol, Sucrose, Acetyl Hexapeptide-8, Glucose Oxidase, Polyacrylamide, Acetyl Carnitine Hcl, Glucose, Caprylic/Capric Triglyceride, C13-14 Isoparaffin, Tocopheryl Acetate, Tetrahexyldecyl Ascorbate, Sodium Pca, Glycosaminoglycans, Urea, Distearyldimonium Chloride, Dipalmitoyl Hydroxyproline, Sodium Hyaluronate, Laureth-7, Lecithin, Trehalose, Polyquaternium-51, Lactoperoxidase, Hydroxypropyl Cyclodextrin, Cyanocobalamin, Pentylene Glycol, Fragrance, Disodium Edta, Bht, Citronellol, Hydroxycitronellal, Geraniol, Linalool, Limonene, Potassium Sorbate, Phenoxyethanol",1,1,1,1,1
4,Moisturizer,IT COSMETICS,Your Skin But Better™ CC+™ C

In [675]:
raw_df.describe()

,Price,Rank,Combination,Dry,Normal,Oily,Sensitive
count,1472.000000,1472.000000,1472.00000,1472.000000,1472.000000,1472.000000,1472.000000
mean,55.584239,4.153261,0.65625,0.614130,0.652174,0.607337,0.513587
std,45.014429,0.633918,0.47512,0.486965,0.476442,0.488509,0.499985
min,3.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,4.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,42.500000,4.300000,1.00000,1.000000,1.000000,1.000000,1.000000
75%,68.000000,4.500000,1.00000,1.000000,1.000000,1.000000,1.000000
max,370.000000,5.000000,1.00000,1.000000,1.000000,1.000000,1.000000


In [676]:
raw_df.isnull().sum()

Label          0
Brand          0
Name           0
Price          0
Rank           0
Ingredients    0
Combination    0
Dry            0
Normal         0
Oily           0
Sensitive      0
dtype: int64

In [677]:
raw_df.columns

Index(['Label', 'Brand', 'Name', 'Price', 'Rank', 'Ingredients', 'Combination',
       'Dry', 'Normal', 'Oily', 'Sensitive'],
      dtype='object')

In [678]:
with_types = raw_df
def add_skin_types_names(row):
    types = []
    if row["Combination"]:
        types.append("combination")
    if row["Dry"]:
        types.append("dry")
    if row["Normal"]:
        types.append("normal")
    if row["Oily"]:
        types.append("oily")
    if row["Sensitive"]:
        types.append("sensitive")

    if not row["Combination"] and not row["Dry"] and not row["Normal"] and not row["Oily"] and not row["Sensitive"]:
        types.append("combination")
        types.append("dry")
        types.append("normal")
        types.append("oily")
        types.append("sensitive")
        types.append("alltypes")
    if row["Combination"] and row["Dry"] and row["Normal"] and row["Oily"]and row["Sensitive"]:
        types.append("alltypes")

    return " ".join(types)

with_types["Skin Types"] = with_types.apply(add_skin_types_names, axis=1)
with_types.head()


,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive,Skin Types
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolatum, Glycerin, Isohexadecane, Microcrystalline Wax, Lanolin Alcohol, Citrus Aurantifolia (Lime) Extract, Sesamum Indicum (Sesame) Seed Oil, Eucalyptus Globulus (Eucalyptus) Leaf Oil, Sesamum Indicum (Sesame) Seed Powder, Medicago Sativa (Alfalfa) Seed Powder, Helianthus Annuus (Sunflower) Seedcake, Prunus Amygdalus Dulcis (Sweet Almond) Seed Meal, Sodium Gluconate, Copper Gluconate, Calcium Gluconate, Magnesium Gluconate, Zinc Gluconate, Magnesium Sulfate, Paraffin, Tocopheryl Succinate, Niacin, Water, Beta-Carotene, Decyl Oleate, Aluminum Distearate, Octyldodecanol, Citric Acid, Cyanocobalamin, Magnesium Stearate, Panthenol, Limonene, Geraniol, Linalool, Hydroxycitronellal, Citronellol, Benzyl Salicylate, Citral, Sodium Benzoate, Alcohol Denat., Fragrance.",1,1,1,1,1,combination dry normal oily sensitive alltypes
1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butylene Glycol, Pentylene Glycol, Water, Sodium Benzoate, Methylparaben, Sorbic Acid.",1,1,1,1,1,combination dry normal oily sensitive alltypes
2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Cetearyl Alcohol, Cetearyl Olivate, Sorbitan Olivate, Sclerocarya Birrea Seed Oil, Bacillus/Soybean/ Folic Acid Ferment Extract, Nymphaea Alba Root Extract, sh-Oligopeptide-1, sh-Oligopeptide-2, sh-Polypeptide-1, sh-Polypeptide-9, sh-Polypeptide-11, Copper Palmitoyl Heptapeptide-14, Heptapeptide-15 Palmitate, Palmitoyl Tetrapeptide-7, Palmitoyl Tripeptide-1, Alanine, Arginine, Glycine, Histidine, Isoleucine, Phenylalanine, Proline, Serine, Threonine, Valine, Acetyl Glutamine, Coconut Alkanes , Coco-Caprylate/Caprate, Sodium Hyaluronate, Aspartic Acid, Linoleic Acid, Linolenic Acid, Lecithin, Butylene Glycol, Polyvinyl Alcohol, Sodium Lactate, Sodium PCA, PCA, Sorbitan Isostearate, Carbomer, Polysorbate 20, Polysorbate 60, Lactic Acid/Glycolic Acid Copolymer, Hydroxyethyl Acrylate/Sodium Acryloyldimethyl Taurate Copolymer, Xanthan Gum, Isomalt, 1,2-Hexanediol, Caprylyl Glycol, Chlorphenesin, Phenoxyethanol, Tocopherol, Sodium Benzoate, Phenylpropanol, Glyceryl Caprylate, Symphytum Officinale Callus Culture Extract.",1,1,1,1,0,combination dry normal oily
3,Moisturizer,LA MER,The Moisturizing Soft Cream,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, Petrolatum, Glyceryl Distearate, Phenyl Trimethicone, Butylene Glycol, Hydrogenated Vegetable Oil, Cholesterol, Butyrospermum Parkii (Shea Butter), Steareth-10, Dimethicone, Glyceryl Stearate Se, Polysilicone-11, Sesamum Indicum (Sesame) Seed Oil, Medicago Sativa (Alfalfa) Seed Powder, Helianthus Annuus (Sunflower) Seedcake, Prunus Amygdalus Dulcis (Sweet Almond) Seed Meal, Eucalyptus Globulus (Eucalyptus) Leaf Oil, Sodium Gluconate, Copper Gluconate, Calcium Gluconate, Magnesium Gluconate, Zinc Gluconate, Tocopheryl Succinate, Niacin, Sesamum Indicum (Sesame) Seed Powder, Water, Citrus Aurantifolia (Lime) Peel Extract, Laminaria Digitata Extract, Crithmum Maritimum Extract, Salicornia Herbacea Extract, Plankton Extract, Chlorella Vulgaris Extract, Glycine Soja (Soybean) Seed Extract, Glycerin, Caffeine, Sea Salt/Maris Sal/Sel Marin, Micrococcus Lysate, Diethylhexyl Succinate, Adenosine Phosphate, Creatine, Hydrolyzed Algin, Isocetyl Stearoyl Stearate, Cetyl Alcohol, Sucrose, Acetyl Hexapeptide-8, Glucose Oxidase, Polyacrylamide, Acetyl Carnitine Hcl, Glucose, Caprylic/Capric Triglyceride, C13-14 Isoparaffin, Tocopheryl Acetate, Tetrahexyldecyl Ascorbate, Sodium Pca, Glycosaminoglycans, Urea, Distearyldimonium Chloride, Dipalmitoyl Hydroxyproline, Sodium Hyaluronate, Laureth-7, Lecithin, Trehalose, Polyquaternium-51, Lactoperoxidase, Hydroxypropyl Cyclodextrin, Cyanocobalamin, Pentylene Glycol, Fragrance, Disodium Edta, Bht, Citronellol, Hydroxycitronella

In [679]:
with_types = with_types.map(lambda content: content.lower() if type(content) == str else content)

In [680]:
with_types["Ingredients"]

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [681]:
def clean_ingredients(text):
    # text = text.lower()
    text =text.replace("\n", " ")
    if "ingredients:" in text:
        text = text.split("ingredients:")[1].strip()
    text = re.sub(r':((?!.*?,).*?)\.', '', text)
    text =text.replace(" -", ", ")
    text = text.replace(" =;", ",")
    if text.startswith("-"):
        text =text.replace("-", "")
    text =text.replace("(", "")
    text =text.replace(")", "")
    text =text.replace("*", "")


    if text.startswith("visit") or text.startswith("#name?"):
        return "No info"
    return text

with_types["Clean Ingredients"] = with_types["Ingredients"].apply(clean_ingredients)
with_types["Clean Ingredients"].head(190)


0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [682]:
def clean_names(text):
    normalized = unicodedata.normalize("NFD", text)
    without_accents = "".join(character for character in normalized if unicodedata.category(character) != "Mn")
    cleaned = re.sub(r"[^a-zA-Z0-9\s]", "", without_accents)
    return cleaned.lower()

In [683]:
with_types["Clean Name"] = with_types["Name"].apply(clean_names)
with_types

,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive,Skin Types,Clean Ingredients,Clean Name
0,moisturizer,la mer,crème de la mer,175,4.1,"algae (seaweed) extract, mineral oil, petrolatum, glycerin, isohexadecane, microcrystalline wax, lanolin alcohol, citrus aurantifolia (lime) extract, sesamum indicum (sesame) seed oil, eucalyptus globulus (eucalyptus) leaf oil, sesamum indicum (sesame) seed powder, medicago sativa (alfalfa) seed powder, helianthus annuus (sunflower) seedcake, prunus amygdalus dulcis (sweet almond) seed meal, sodium gluconate, copper gluconate, calcium gluconate, magnesium gluconate, zinc gluconate, magnesium sulfate, paraffin, tocopheryl succinate, niacin, water, beta-carotene, decyl oleate, aluminum distearate, octyldodecanol, citric acid, cyanocobalamin, magnesium stearate, panthenol, limonene, geraniol, linalool, hydroxycitronellal, citronellol, benzyl salicylate, citral, sodium benzoate, alcohol denat., fragrance.",1,1,1,1,1,combination dry normal oily sensitive alltypes,"algae seaweed extract, mineral oil, petrolatum, glycerin, isohexadecane, microcrystalline wax, lanolin alcohol, citrus aurantifolia lime extract, sesamum indicum sesame seed oil, eucalyptus globulus eucalyptus leaf oil, sesamum indicum sesame seed powder, medicago sativa alfalfa seed powder, helianthus annuus sunflower seedcake, prunus amygdalus dulcis sweet almond seed meal, sodium gluconate, copper gluconate, calcium gluconate, magnesium gluconate, zinc gluconate, magnesium sulfate, paraffin, tocopheryl succinate, niacin, water, beta-carotene, decyl oleate, aluminum distearate, octyldodecanol, citric acid, cyanocobalamin, magnesium stearate, panthenol, limonene, geraniol, linalool, hydroxycitronellal, citronellol, benzyl salicylate, citral, sodium benzoate, alcohol denat., fragrance.",creme de la mer
1,moisturizer,sk-ii,facial treatment essence,179,4.1,"galactomyces ferment filtrate (pitera), butylene glycol, pentylene glycol, water, sodium benzoate, methylparaben, sorbic acid.",1,1,1,1,1,combination dry normal oily sensitive alltypes,"galactomyces ferment filtrate pitera, butylene glycol, pentylene glycol, water, sodium benzoate, methylparaben, sorbic acid.",facial treatment essence
2,moisturizer,drunk elephant,protini™ polypeptide cream,68,4.4,"water, dicaprylyl carbonate, glycerin, cetearyl alcohol, cetearyl olivate, sorbitan olivate, sclerocarya birrea seed oil, bacillus/soybean/ folic acid ferment extract, nymphaea alba root extract, sh-oligopeptide-1, sh-oligopeptide-2, sh-polypeptide-1, sh-polypeptide-9, sh-polypeptide-11, copper palmitoyl heptapeptide-14, heptapeptide-15 palmitate, palmitoyl tetrapeptide-7, palmitoyl tripeptide-1, alanine, arginine, glycine, histidine, isoleucine, phenylalanine, proline, serine, threonine, valine, acetyl glutamine, coconut alkanes , coco-caprylate/caprate, sodium hyaluronate, aspartic acid, linoleic acid, linolenic acid, lecithin, butylene glycol, polyvinyl alcohol, sodium lactate, sodium pca, pca, sorbitan isostearate, carbomer, polysorbate 20, polysorbate 60, lactic acid/glycolic acid copolymer, hydroxyethyl acrylate/sodium acryloyldimethyl taurate copolymer, xanthan gum, isomalt, 1,2-hexanediol, caprylyl glycol, chlorphenesin, phenoxyethanol, tocopherol, sodium benzoate, phenylpropanol, glyceryl caprylate, symphytum officinale callus culture extract.",1,1,1,1,0,combination dry normal oily,"water, dicaprylyl carbonate, glycerin, cetearyl alcohol, cetearyl olivate, sorbitan olivate, sclerocarya birrea seed oil, bacillus/soybean/ folic acid ferment extract, nymphaea alba root extract, sh-oligopeptide-1, sh-oligopeptide-2, sh-polypeptide-1, sh-polypeptide-9, sh-polypeptide-11, copper palmitoyl heptapeptide-14, heptapeptide-15 palmitate, palmitoyl tetrapeptide-7, palmitoyl tripeptide-1, alanine, arginine, glycine, histidine, isoleucine, phenylalanine, proline, serine, threonine, valine, acetyl glutamine, coconut alkanes , coco-caprylate/caprate, sodium hyaluronate, asp

In [684]:
with_types.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 0 to 1471
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Label              1472 non-null   object 
 1   Brand              1472 non-null   object 
 2   Name               1472 non-null   object 
 3   Price              1472 non-null   int64  
 4   Rank               1472 non-null   float64
 5   Ingredients        1472 non-null   object 
 6   Combination        1472 non-null   int64  
 7   Dry                1472 non-null   int64  
 8   Normal             1472 non-null   int64  
 9   Oily               1472 non-null   int64  
 10  Sensitive          1472 non-null   int64  
 11  Skin Types         1472 non-null   object 
 12  Clean Ingredients  1472 non-null   object 
 13  Clean Name         1472 non-null   object 
dtypes: float64(1), int64(6), object(7)
memory usage: 161.1+ KB


In [685]:
dataframe = with_types[["Label", "Brand",  "Clean Name", "Clean Ingredients", "Skin Types","Rank"]].copy()
dataframe.head()

,Label,Brand,Clean Name,Clean Ingredients,Skin Types,Rank
0,moisturizer,la mer,creme de la mer,"algae seaweed extract, mineral oil, petrolatum, glycerin, isohexadecane, microcrystalline wax, lanolin alcohol, citrus aurantifolia lime extract, sesamum indicum sesame seed oil, eucalyptus globulus eucalyptus leaf oil, sesamum indicum sesame seed powder, medicago sativa alfalfa seed powder, helianthus annuus sunflower seedcake, prunus amygdalus dulcis sweet almond seed meal, sodium gluconate, copper gluconate, calcium gluconate, magnesium gluconate, zinc gluconate, magnesium sulfate, paraffin, tocopheryl succinate, niacin, water, beta-carotene, decyl oleate, aluminum distearate, octyldodecanol, citric acid, cyanocobalamin, magnesium stearate, panthenol, limonene, geraniol, linalool, hydroxycitronellal, citronellol, benzyl salicylate, citral, sodium benzoate, alcohol denat., fragrance.",combination dry normal oily sensitive alltypes,4.1
1,moisturizer,sk-ii,facial treatment essence,"galactomyces ferment filtrate pitera, butylene glycol, pentylene glycol, water, sodium benzoate, methylparaben, sorbic acid.",combination dry normal oily sensitive alltypes,4.1
2,moisturizer,drunk elephant,protini polypeptide cream,"water, dicaprylyl carbonate, glycerin, cetearyl alcohol, cetearyl olivate, sorbitan olivate, sclerocarya birrea seed oil, bacillus/soybean/ folic acid ferment extract, nymphaea alba root extract, sh-oligopeptide-1, sh-oligopeptide-2, sh-polypeptide-1, sh-polypeptide-9, sh-polypeptide-11, copper palmitoyl heptapeptide-14, heptapeptide-15 palmitate, palmitoyl tetrapeptide-7, palmitoyl tripeptide-1, alanine, arginine, glycine, histidine, isoleucine, phenylalanine, proline, serine, threonine, valine, acetyl glutamine, coconut alkanes , coco-caprylate/caprate, sodium hyaluronate, aspartic acid, linoleic acid, linolenic acid, lecithin, butylene glycol, polyvinyl alcohol, sodium lactate, sodium pca, pca, sorbitan isostearate, carbomer, polysorbate 20, polysorbate 60, lactic acid/glycolic acid copolymer, hydroxyethyl acrylate/sodium acryloyldimethyl taurate copolymer, xanthan gum, isomalt, 1,2-hexanediol, caprylyl glycol, chlorphenesin, phenoxyethanol, tocopherol, sodium benzoate, phenylpropanol, glyceryl caprylate, symphytum officinale callus culture extract.",combination dry normal oily,4.4
3,moisturizer,la mer,the moisturizing soft cream,"algae seaweed extract, cyclopentasiloxane, petrolatum, glyceryl distearate, phenyl trimethicone, butylene glycol, hydrogenated vegetable oil, cholesterol, butyrospermum parkii shea butter, steareth-10, dimethicone, glyceryl stearate se, polysilicone-11, sesamum indicum sesame seed oil, medicago sativa alfalfa seed powder, helianthus annuus sunflower seedcake, prunus amygdalus dulcis sweet almond seed meal, eucalyptus globulus eucalyptus leaf oil, sodium gluconate, copper gluconate, calcium gluconate, magnesium gluconate, zinc gluconate, tocopheryl succinate, niacin, sesamum indicum sesame seed powder, water, citrus aurantifolia lime peel extract, laminaria digitata extract, crithmum maritimum extract, salicornia herbacea extract, plankton extract, chlorella vulgaris extract, glycine soja soybean seed extract, glycerin, caffeine, sea salt/maris sal/sel marin, micrococcus lysate, diethylhexyl succinate, adenosine phosphate, creatine, hydrolyzed algin, isocetyl stearoyl stearate, cetyl alcohol, sucrose, acetyl hexapeptide-8, glucose oxidase, polyacrylamide, acetyl carnitine hcl, glucose, caprylic/capric triglyceride, c13-14 isoparaffin, tocopheryl acetate, tetrahexyldecyl ascorbate, sodium pca, glycosaminoglycans, urea, distearyldimonium chloride, dipalmitoyl hydroxyproline, sodium hyaluronate, laureth-7, lecithin, trehalose, polyquaternium-51, lactoperoxidase, hydroxypropyl cyclodextrin, cyanocobalamin, pentylene glycol, fragrance, disodium edta, bht, citronellol, hydroxycitronellal, geraniol, linalool, limonene, potassium sorbate, phenoxyethanol",combination dry normal oily sensitive alltypes,3.8
4

In [686]:
query = "oily"
dataframe[ dataframe["Skin Types"].str.contains(query) ].head()

,Label,Brand,Clean Name,Clean Ingredients,Skin Types,Rank
0,moisturizer,la mer,creme de la mer,"algae seaweed extract, mineral oil, petrolatum, glycerin, isohexadecane, microcrystalline wax, lanolin alcohol, citrus aurantifolia lime extract, sesamum indicum sesame seed oil, eucalyptus globulus eucalyptus leaf oil, sesamum indicum sesame seed powder, medicago sativa alfalfa seed powder, helianthus annuus sunflower seedcake, prunus amygdalus dulcis sweet almond seed meal, sodium gluconate, copper gluconate, calcium gluconate, magnesium gluconate, zinc gluconate, magnesium sulfate, paraffin, tocopheryl succinate, niacin, water, beta-carotene, decyl oleate, aluminum distearate, octyldodecanol, citric acid, cyanocobalamin, magnesium stearate, panthenol, limonene, geraniol, linalool, hydroxycitronellal, citronellol, benzyl salicylate, citral, sodium benzoate, alcohol denat., fragrance.",combination dry normal oily sensitive alltypes,4.1
1,moisturizer,sk-ii,facial treatment essence,"galactomyces ferment filtrate pitera, butylene glycol, pentylene glycol, water, sodium benzoate, methylparaben, sorbic acid.",combination dry normal oily sensitive alltypes,4.1
2,moisturizer,drunk elephant,protini polypeptide cream,"water, dicaprylyl carbonate, glycerin, cetearyl alcohol, cetearyl olivate, sorbitan olivate, sclerocarya birrea seed oil, bacillus/soybean/ folic acid ferment extract, nymphaea alba root extract, sh-oligopeptide-1, sh-oligopeptide-2, sh-polypeptide-1, sh-polypeptide-9, sh-polypeptide-11, copper palmitoyl heptapeptide-14, heptapeptide-15 palmitate, palmitoyl tetrapeptide-7, palmitoyl tripeptide-1, alanine, arginine, glycine, histidine, isoleucine, phenylalanine, proline, serine, threonine, valine, acetyl glutamine, coconut alkanes , coco-caprylate/caprate, sodium hyaluronate, aspartic acid, linoleic acid, linolenic acid, lecithin, butylene glycol, polyvinyl alcohol, sodium lactate, sodium pca, pca, sorbitan isostearate, carbomer, polysorbate 20, polysorbate 60, lactic acid/glycolic acid copolymer, hydroxyethyl acrylate/sodium acryloyldimethyl taurate copolymer, xanthan gum, isomalt, 1,2-hexanediol, caprylyl glycol, chlorphenesin, phenoxyethanol, tocopherol, sodium benzoate, phenylpropanol, glyceryl caprylate, symphytum officinale callus culture extract.",combination dry normal oily,4.4
3,moisturizer,la mer,the moisturizing soft cream,"algae seaweed extract, cyclopentasiloxane, petrolatum, glyceryl distearate, phenyl trimethicone, butylene glycol, hydrogenated vegetable oil, cholesterol, butyrospermum parkii shea butter, steareth-10, dimethicone, glyceryl stearate se, polysilicone-11, sesamum indicum sesame seed oil, medicago sativa alfalfa seed powder, helianthus annuus sunflower seedcake, prunus amygdalus dulcis sweet almond seed meal, eucalyptus globulus eucalyptus leaf oil, sodium gluconate, copper gluconate, calcium gluconate, magnesium gluconate, zinc gluconate, tocopheryl succinate, niacin, sesamum indicum sesame seed powder, water, citrus aurantifolia lime peel extract, laminaria digitata extract, crithmum maritimum extract, salicornia herbacea extract, plankton extract, chlorella vulgaris extract, glycine soja soybean seed extract, glycerin, caffeine, sea salt/maris sal/sel marin, micrococcus lysate, diethylhexyl succinate, adenosine phosphate, creatine, hydrolyzed algin, isocetyl stearoyl stearate, cetyl alcohol, sucrose, acetyl hexapeptide-8, glucose oxidase, polyacrylamide, acetyl carnitine hcl, glucose, caprylic/capric triglyceride, c13-14 isoparaffin, tocopheryl acetate, tetrahexyldecyl ascorbate, sodium pca, glycosaminoglycans, urea, distearyldimonium chloride, dipalmitoyl hydroxyproline, sodium hyaluronate, laureth-7, lecithin, trehalose, polyquaternium-51, lactoperoxidase, hydroxypropyl cyclodextrin, cyanocobalamin, pentylene glycol, fragrance, disodium edta, bht, citronellol, hydroxycitronellal, geraniol, linalool, limonene, potassium sorbate, phenoxyethanol",combination dry normal oily sensitive alltypes,3.8
4

In [687]:
query = "combination oily"
dataframe[ dataframe["Skin Types"].str.strip() == query ].head()

,Label,Brand,Clean Name,Clean Ingredients,Skin Types,Rank
42,moisturizer,clinique,dramatically different moisturizing gel,"water , dimethicone , isododecane , butylene glycol , bis-peg-18 methyl ether dimethyl silane , glycerin , laminaria saccharina extract , polygonum cuspidatum root extract , saccharomyces lysate extract , cucumis sativus cucumber fruit extract , hordeum vulgare barley extract/extrait d'orge , helianthus annuus sunflower seedcake , caffeine , trehalose , sodium hyaluronate , tocopheryl acetate , chamomilla recutita matricaria , polysilicone-11 , silica , propylene glycol dicaprate , oleth-10 , lactobacillus ferment , laureth-23 , laureth-4 , ammonium acryloyldimethyltaurate/vp copolymer , carbomer , caprylyl glycol , hexylene glycol , tromethamine , disodium edta , phenoxyethanol , potassium sorbate , yellow 5 ci 19140 , red 4 ci 14700 , yellow 6 ci 15985",combination oily,4.5
150,moisturizer,murad,oilcontrol mattifier spf 15 pa,"water, cyclopentasiloxane, bis-peg/ppg-14/14 dimethicone, dimethicone crosspolymer, corn starch modified, butylene glycol, chitosan , ascorbic acid, polyglyceryl-3 polydimethylsiloxyethyl dimethicone, triethoxysilylethyl polydimethylsiloxyethyl hexyl dimethicone, farnesyl acetate, panthenyl triacetate, argania spinosa kernel extract, serenoa serrulata fruit extract, sesamum indicum sesame seed extract, punica granatum seed oil, sodium pca, betaine, sorbitol, glycine, alanine, proline, serine, threonine, arginine, lysine, glutamic acid, capryloyl glycine, sarcosine, cinnamomum zeylanicum bark extract, aluminum starch octenylsuccinate, acrylates copolymer, magnesium carbonate, melaleuca alternifolia tea tree leaf oil, tocopheryl acetate, lauryl methacrylate/glycol dimethacrylate crosspolymer, dicaprylyl ether, niacinamide, yeast extract faex, aesculus hippocastanum horse chestnut seed extract, titanium dioxide, algae extract, ammonium glycyrrhizate, panthenol, zinc gluconate, caffeine, xanthan gum, acrylates crosspolymer, polyglyceryl-2 dipolyhydroxystearate, biotin, salix alba willow bark extract, aminomethyl propanol, disodium edta, sodium chloride, propyl gallate, phenoxyethanol, chlorphenesin.",combination oily,4.2
151,moisturizer,olehenriksen,counter balance oil control hydrator,"water, butylene glycol, glycerin, caprylic/capric triglyceride, dimethicone, acrylates/c10-30 alykyl acrylate crosspolymer, methyl methacrylate crosspolymer, sodium hydroxide, polysorbate 20, linoleic acid, phenoxyethanol, glycolic acid, lactic acid, melia azadirachta seed oil, avena sativa oat kernel flour, methyl gluceth-20, caprylyl glycol, linolenic acid, salicylic acid, mentha piperita peppermint oil, enantia chlorantha bark extract, macrocystis pyrifera kelp extract, eucalyptus globulus leaf extract, chondrus crispus carrageenan extract, camellia sinensis leaf extract, tocopheryl acetate, hexylene glycol, ethylhexylglycerin, dimethicone crosspolymer, bisabolol, allantoin, phytic acid, fragrance, saccarhide isomerate, rosa canina fruit extract, tocopherol, sodium hyaluronate, oleanolic acid, limonene.",combination oily,4.3
163,moisturizer,clinique,acne solutions bb cream broad spectrum spf 40,"octinoxate 7.5% , octisalate 2.0% , titanium dioxide 1.6%water , methyl trimethicone , alcohol , ethylhexyl palmitate , peg-9 polydimethylsiloxyethyl dimethicone , propanediol , glycerin , glyceryl caprylate , peg-3 dimethicone , dimethicone , tocopheryl acetate , lecithin , triethoxycaprylylsilane , stearyl glycyrrhetinate , sorbitan sesquiisostearate , sodium hyaluronate , pyridoxine hcl , stearic acid , hesperidin methyl chalcone , palmitoyl tetrapeptide-7 , sodium chloride , aluminum hydroxide , steareth-20 , dipeptide-2 , silica , sodium citrate , phenoxyethanol , [+/- mica , iron oxides ci 77491 , iron oxides ci 77492 , iron oxides ci 77499 , titanium dioxide ci 77891]",combination oily,4.1
194,moisturizer,clinique,limited edition dramatically different moisturizing gel,"water, dimethicone, isododecane,

In [688]:
dataframe["Skin Types"] = dataframe["Skin Types"].map(lambda types: types.split(" "))
dataframe.head()

,Label,Brand,Clean Name,Clean Ingredients,Skin Types,Rank
0,moisturizer,la mer,creme de la mer,"algae seaweed extract, mineral oil, petrolatum, glycerin, isohexadecane, microcrystalline wax, lanolin alcohol, citrus aurantifolia lime extract, sesamum indicum sesame seed oil, eucalyptus globulus eucalyptus leaf oil, sesamum indicum sesame seed powder, medicago sativa alfalfa seed powder, helianthus annuus sunflower seedcake, prunus amygdalus dulcis sweet almond seed meal, sodium gluconate, copper gluconate, calcium gluconate, magnesium gluconate, zinc gluconate, magnesium sulfate, paraffin, tocopheryl succinate, niacin, water, beta-carotene, decyl oleate, aluminum distearate, octyldodecanol, citric acid, cyanocobalamin, magnesium stearate, panthenol, limonene, geraniol, linalool, hydroxycitronellal, citronellol, benzyl salicylate, citral, sodium benzoate, alcohol denat., fragrance.","[combination, dry, normal, oily, sensitive, alltypes]",4.1
1,moisturizer,sk-ii,facial treatment essence,"galactomyces ferment filtrate pitera, butylene glycol, pentylene glycol, water, sodium benzoate, methylparaben, sorbic acid.","[combination, dry, normal, oily, sensitive, alltypes]",4.1
2,moisturizer,drunk elephant,protini polypeptide cream,"water, dicaprylyl carbonate, glycerin, cetearyl alcohol, cetearyl olivate, sorbitan olivate, sclerocarya birrea seed oil, bacillus/soybean/ folic acid ferment extract, nymphaea alba root extract, sh-oligopeptide-1, sh-oligopeptide-2, sh-polypeptide-1, sh-polypeptide-9, sh-polypeptide-11, copper palmitoyl heptapeptide-14, heptapeptide-15 palmitate, palmitoyl tetrapeptide-7, palmitoyl tripeptide-1, alanine, arginine, glycine, histidine, isoleucine, phenylalanine, proline, serine, threonine, valine, acetyl glutamine, coconut alkanes , coco-caprylate/caprate, sodium hyaluronate, aspartic acid, linoleic acid, linolenic acid, lecithin, butylene glycol, polyvinyl alcohol, sodium lactate, sodium pca, pca, sorbitan isostearate, carbomer, polysorbate 20, polysorbate 60, lactic acid/glycolic acid copolymer, hydroxyethyl acrylate/sodium acryloyldimethyl taurate copolymer, xanthan gum, isomalt, 1,2-hexanediol, caprylyl glycol, chlorphenesin, phenoxyethanol, tocopherol, sodium benzoate, phenylpropanol, glyceryl caprylate, symphytum officinale callus culture extract.","[combination, dry, normal, oily]",4.4
3,moisturizer,la mer,the moisturizing soft cream,"algae seaweed extract, cyclopentasiloxane, petrolatum, glyceryl distearate, phenyl trimethicone, butylene glycol, hydrogenated vegetable oil, cholesterol, butyrospermum parkii shea butter, steareth-10, dimethicone, glyceryl stearate se, polysilicone-11, sesamum indicum sesame seed oil, medicago sativa alfalfa seed powder, helianthus annuus sunflower seedcake, prunus amygdalus dulcis sweet almond seed meal, eucalyptus globulus eucalyptus leaf oil, sodium gluconate, copper gluconate, calcium gluconate, magnesium gluconate, zinc gluconate, tocopheryl succinate, niacin, sesamum indicum sesame seed powder, water, citrus aurantifolia lime peel extract, laminaria digitata extract, crithmum maritimum extract, salicornia herbacea extract, plankton extract, chlorella vulgaris extract, glycine soja soybean seed extract, glycerin, caffeine, sea salt/maris sal/sel marin, micrococcus lysate, diethylhexyl succinate, adenosine phosphate, creatine, hydrolyzed algin, isocetyl stearoyl stearate, cetyl alcohol, sucrose, acetyl hexapeptide-8, glucose oxidase, polyacrylamide, acetyl carnitine hcl, glucose, caprylic/capric triglyceride, c13-14 isoparaffin, tocopheryl acetate, tetrahexyldecyl ascorbate, sodium pca, glycosaminoglycans, urea, distearyldimonium chloride, dipalmitoyl hydroxyproline, sodium hyaluronate, laureth-7, lecithin, trehalose, polyquaternium-51, lactoperoxidase, hydroxypropyl cyclodextrin, cyanocobalamin, pentylene glycol, fragrance, disodium edta, bht, citronellol, hydroxycitronellal, geraniol, linalool, limonene, potassium sorbate, phenoxyethanol","[combination, dry, normal,

In [689]:
dataframe["keywords"] = ""

In [690]:
# nltk.download('stopwords')

In [691]:
for index, row in dataframe.iterrows():
    ingredients = row["Clean Ingredients"]

    r = Rake()

    r.extract_keywords_from_text(ingredients)
    keyword_dict_scores = r.get_word_degrees()
    dataframe.at[index, "keywords"] = list(keyword_dict_scores.keys())

In [692]:
dataframe[["Clean Ingredients", "keywords"]]

,Clean Ingredients,keywords
0,"algae seaweed extract, mineral oil, petrolatum, glycerin, isohexadecane, microcrystalline wax, lanolin alcohol, citrus aurantifolia lime extract, sesamum indicum sesame seed oil, eucalyptus globulus eucalyptus leaf oil, sesamum indicum sesame seed powder, medicago sativa alfalfa seed powder, helianthus annuus sunflower seedcake, prunus amygdalus dulcis sweet almond seed meal, sodium gluconate, copper gluconate, calcium gluconate, magnesium gluconate, zinc gluconate, magnesium sulfate, paraffin, tocopheryl succinate, niacin, water, beta-carotene, decyl oleate, aluminum distearate, octyldodecanol, citric acid, cyanocobalamin, magnesium stearate, panthenol, limonene, geraniol, linalool, hydroxycitronellal, citronellol, benzyl salicylate, citral, sodium benzoate, alcohol denat., fragrance.","[algae, seaweed, extract, mineral, oil, petrolatum, glycerin, isohexadecane, microcrystalline, wax, lanolin, alcohol, citrus, aurantifolia, lime, sesamum, indicum, sesame, seed, eucalyptus, globulus, leaf, powder, medicago, sativa, alfalfa, helianthus, annuus, sunflower, seedcake, prunus, amygdalus, dulcis, sweet, almond, meal, sodium, gluconate, copper, calcium, magnesium, zinc, sulfate, paraffin, tocopheryl, succinate, niacin, water, beta, carotene, decyl, oleate, aluminum, distearate, octyldodecanol, citric, acid, cyanocobalamin, stearate, panthenol, limonene, geraniol, linalool, hydroxycitronellal, citronellol, benzyl, salicylate, citral, benzoate, denat, .,, fragrance]"
1,"galactomyces ferment filtrate pitera, butylene glycol, pentylene glycol, water, sodium benzoate, methylparaben, sorbic acid.","[galactomyces, ferment, filtrate, pitera, butylene, glycol, pentylene, water, sodium, benzoate, methylparaben, sorbic, acid]"
2,"water, dicaprylyl carbonate, glycerin, cetearyl alcohol, cetearyl olivate, sorbitan olivate, sclerocarya birrea seed oil, bacillus/soybean/ folic acid ferment extract, nymphaea alba root extract, sh-oligopeptide-1, sh-oligopeptide-2, sh-polypeptide-1, sh-polypeptide-9, sh-polypeptide-11, copper palmitoyl heptapeptide-14, heptapeptide-15 palmitate, palmitoyl tetrapeptide-7, palmitoyl tripeptide-1, alanine, arginine, glycine, histidine, isoleucine, phenylalanine, proline, serine, threonine, valine, acetyl glutamine, coconut alkanes , coco-caprylate/caprate, sodium hyaluronate, aspartic acid, linoleic acid, linolenic acid, lecithin, butylene glycol, polyvinyl alcohol, sodium lactate, sodium pca, pca, sorbitan isostearate, carbomer, polysorbate 20, polysorbate 60, lactic acid/glycolic acid copolymer, hydroxyethyl acrylate/sodium acryloyldimethyl taurate copolymer, xanthan gum, isomalt, 1,2-hexanediol, caprylyl glycol, chlorphenesin, phenoxyethanol, tocopherol, sodium benzoate, phenylpropanol, glyceryl caprylate, symphytum officinale callus culture extract.","[water, dicaprylyl, carbonate, glycerin, cetearyl, alcohol, olivate, sorbitan, sclerocarya, birrea, seed, oil, bacillus, soybean, folic, acid, ferment, extract, nymphaea, alba, root, sh, oligopeptide, 1, 2, polypeptide, 9, 11, copper, palmitoyl, heptapeptide, 14, 15, palmitate, tetrapeptide, 7, tripeptide, alanine, arginine, glycine, histidine, isoleucine, phenylalanine, proline, serine, threonine, valine, acetyl, glutamine, coconut, alkanes, coco, caprylate, caprate, sodium, hyaluronate, aspartic, linoleic, linolenic, lecithin, butylene, glycol, polyvinyl, lactate, pca, isostearate, carbomer, polysorbate, 20, 60, lactic, glycolic, copolymer, hydroxyethyl, acrylate, acryloyldimethyl, taurate, xanthan, gum, isomalt, hexanediol, caprylyl, chlorphenesin, phenoxyethanol, tocopherol, benzoate, phenylpropanol, glyceryl, symphytum, officinale, callus, culture]"
3,"algae seaweed extract, cyclopentasiloxane, petrolatum, glyceryl distearate, phenyl trimethicone, butylene glycol, hydrogenated vegetable oil, cholesterol, butyrospermum parkii shea butter, steareth-10, dimethicone, glyceryl stearate se, polysilicone-11, sesamum indicum sesame seed oil,

In [693]:
dataframe[dataframe["Clean Name"].duplicated()]

,Label,Brand,Clean Name,Clean Ingredients,Skin Types,Rank,keywords


In [694]:
dataframe  = dataframe.set_index("Clean Name")

In [695]:
dataframe

,Label,Brand,Clean Ingredients,Skin Types,Rank,keywords
Clean Name,,,,,,
creme de la mer,moisturizer,la mer,"algae seaweed extract, mineral oil, petrolatum, glycerin, isohexadecane, microcrystalline wax, lanolin alcohol, citrus aurantifolia lime extract, sesamum indicum sesame seed oil, eucalyptus globulus eucalyptus leaf oil, sesamum indicum sesame seed powder, medicago sativa alfalfa seed powder, helianthus annuus sunflower seedcake, prunus amygdalus dulcis sweet almond seed meal, sodium gluconate, copper gluconate, calcium gluconate, magnesium gluconate, zinc gluconate, magnesium sulfate, paraffin, tocopheryl succinate, niacin, water, beta-carotene, decyl oleate, aluminum distearate, octyldodecanol, citric acid, cyanocobalamin, magnesium stearate, panthenol, limonene, geraniol, linalool, hydroxycitronellal, citronellol, benzyl salicylate, citral, sodium benzoate, alcohol denat., fragrance.","[combination, dry, normal, oily, sensitive, alltypes]",4.1,"[algae, seaweed, extract, mineral, oil, petrolatum, glycerin, isohexadecane, microcrystalline, wax, lanolin, alcohol, citrus, aurantifolia, lime, sesamum, indicum, sesame, seed, eucalyptus, globulus, leaf, powder, medicago, sativa, alfalfa, helianthus, annuus, sunflower, seedcake, prunus, amygdalus, dulcis, sweet, almond, meal, sodium, gluconate, copper, calcium, magnesium, zinc, sulfate, paraffin, tocopheryl, succinate, niacin, water, beta, carotene, decyl, oleate, aluminum, distearate, octyldodecanol, citric, acid, cyanocobalamin, stearate, panthenol, limonene, geraniol, linalool, hydroxycitronellal, citronellol, benzyl, salicylate, citral, benzoate, denat, .,, fragrance]"
facial treatment essence,moisturizer,sk-ii,"galactomyces ferment filtrate pitera, butylene glycol, pentylene glycol, water, sodium benzoate, methylparaben, sorbic acid.","[combination, dry, normal, oily, sensitive, alltypes]",4.1,"[galactomyces, ferment, filtrate, pitera, butylene, glycol, pentylene, water, sodium, benzoate, methylparaben, sorbic, acid]"
protini polypeptide cream,moisturizer,drunk elephant,"water, dicaprylyl carbonate, glycerin, cetearyl alcohol, cetearyl olivate, sorbitan olivate, sclerocarya birrea seed oil, bacillus/soybean/ folic acid ferment extract, nymphaea alba root extract, sh-oligopeptide-1, sh-oligopeptide-2, sh-polypeptide-1, sh-polypeptide-9, sh-polypeptide-11, copper palmitoyl heptapeptide-14, heptapeptide-15 palmitate, palmitoyl tetrapeptide-7, palmitoyl tripeptide-1, alanine, arginine, glycine, histidine, isoleucine, phenylalanine, proline, serine, threonine, valine, acetyl glutamine, coconut alkanes , coco-caprylate/caprate, sodium hyaluronate, aspartic acid, linoleic acid, linolenic acid, lecithin, butylene glycol, polyvinyl alcohol, sodium lactate, sodium pca, pca, sorbitan isostearate, carbomer, polysorbate 20, polysorbate 60, lactic acid/glycolic acid copolymer, hydroxyethyl acrylate/sodium acryloyldimethyl taurate copolymer, xanthan gum, isomalt, 1,2-hexanediol, caprylyl glycol, chlorphenesin, phenoxyethanol, tocopherol, sodium benzoate, phenylpropanol, glyceryl caprylate, symphytum officinale callus culture extract.","[combination, dry, normal, oily]",4.4,"[water, dicaprylyl, carbonate, glycerin, cetearyl, alcohol, olivate, sorbitan, sclerocarya, birrea, seed, oil, bacillus, soybean, folic, acid, ferment, extract, nymphaea, alba, root, sh, oligopeptide, 1, 2, polypeptide, 9, 11, copper, palmitoyl, heptapeptide, 14, 15, palmitate, tetrapeptide, 7, tripeptide, alanine, arginine, glycine, histidine, isoleucine, phenylalanine, proline, serine, threonine, valine, acetyl, glutamine, coconut, alkanes, coco, caprylate, caprate, sodium, hyaluronate, aspartic, linoleic, linolenic, lecithin, butylene, glycol, polyvinyl, lactate, pca, isostearate, carbomer, polysorbate, 20, 60, lactic, glycolic, copolymer, hydroxyethyl, acrylate, acryloyldimethyl, taurate, xanthan, gum, isomalt, hexanediol, caprylyl, chlorphenesin, phenoxyethanol, tocopherol, benzoate, phenylpropanol, glyceryl, s

In [696]:
dataframe.drop("Clean Ingredients", axis=1, inplace=True)

In [697]:
dataframe.columns

Index(['Label', 'Brand', 'Skin Types', 'Rank', 'keywords'], dtype='object')

In [698]:
without_rank = dataframe[['Label',  'Skin Types',  'keywords']]
columns  = without_rank.columns
dataframe["bow"] = ""

for index, row in dataframe.iterrows():
    words = ""
    for column in columns:
        if isinstance(row[column], str):
            words = words + row[column] + " "
        if isinstance(row[column], list):
            words = words + " ".join(row[column])+ " "
        dataframe.at[index, "bow"] = words

In [699]:
dataframe.head(20)

,Label,Brand,Skin Types,Rank,keywords,bow
Clean Name,,,,,,
creme de la mer,moisturizer,la mer,"[combination, dry, normal, oily, sensitive, alltypes]",4.1,"[algae, seaweed, extract, mineral, oil, petrolatum, glycerin, isohexadecane, microcrystalline, wax, lanolin, alcohol, citrus, aurantifolia, lime, sesamum, indicum, sesame, seed, eucalyptus, globulus, leaf, powder, medicago, sativa, alfalfa, helianthus, annuus, sunflower, seedcake, prunus, amygdalus, dulcis, sweet, almond, meal, sodium, gluconate, copper, calcium, magnesium, zinc, sulfate, paraffin, tocopheryl, succinate, niacin, water, beta, carotene, decyl, oleate, aluminum, distearate, octyldodecanol, citric, acid, cyanocobalamin, stearate, panthenol, limonene, geraniol, linalool, hydroxycitronellal, citronellol, benzyl, salicylate, citral, benzoate, denat, .,, fragrance]","moisturizer combination dry normal oily sensitive alltypes algae seaweed extract mineral oil petrolatum glycerin isohexadecane microcrystalline wax lanolin alcohol citrus aurantifolia lime sesamum indicum sesame seed eucalyptus globulus leaf powder medicago sativa alfalfa helianthus annuus sunflower seedcake prunus amygdalus dulcis sweet almond meal sodium gluconate copper calcium magnesium zinc sulfate paraffin tocopheryl succinate niacin water beta carotene decyl oleate aluminum distearate octyldodecanol citric acid cyanocobalamin stearate panthenol limonene geraniol linalool hydroxycitronellal citronellol benzyl salicylate citral benzoate denat ., fragrance"
facial treatment essence,moisturizer,sk-ii,"[combination, dry, normal, oily, sensitive, alltypes]",4.1,"[galactomyces, ferment, filtrate, pitera, butylene, glycol, pentylene, water, sodium, benzoate, methylparaben, sorbic, acid]",moisturizer combination dry normal oily sensitive alltypes galactomyces ferment filtrate pitera butylene glycol pentylene water sodium benzoate methylparaben sorbic acid
protini polypeptide cream,moisturizer,drunk elephant,"[combination, dry, normal, oily]",4.4,"[water, dicaprylyl, carbonate, glycerin, cetearyl, alcohol, olivate, sorbitan, sclerocarya, birrea, seed, oil, bacillus, soybean, folic, acid, ferment, extract, nymphaea, alba, root, sh, oligopeptide, 1, 2, polypeptide, 9, 11, copper, palmitoyl, heptapeptide, 14, 15, palmitate, tetrapeptide, 7, tripeptide, alanine, arginine, glycine, histidine, isoleucine, phenylalanine, proline, serine, threonine, valine, acetyl, glutamine, coconut, alkanes, coco, caprylate, caprate, sodium, hyaluronate, aspartic, linoleic, linolenic, lecithin, butylene, glycol, polyvinyl, lactate, pca, isostearate, carbomer, polysorbate, 20, 60, lactic, glycolic, copolymer, hydroxyethyl, acrylate, acryloyldimethyl, taurate, xanthan, gum, isomalt, hexanediol, caprylyl, chlorphenesin, phenoxyethanol, tocopherol, benzoate, phenylpropanol, glyceryl, symphytum, officinale, callus, culture]",moisturizer combination dry normal oily water dicaprylyl carbonate glycerin cetearyl alcohol olivate sorbitan sclerocarya birrea seed oil bacillus soybean folic acid ferment extract nymphaea alba root sh oligopeptide 1 2 polypeptide 9 11 copper palmitoyl heptapeptide 14 15 palmitate tetrapeptide 7 tripeptide alanine arginine glycine histidine isoleucine phenylalanine proline serine threonine valine acetyl glutamine coconut alkanes coco caprylate caprate sodium hyaluronate aspartic linoleic linolenic lecithin butylene glycol polyvinyl lactate pca isostearate carbomer polysorbate 20 60 lactic glycolic copolymer hydroxyethyl acrylate acryloyldimethyl taurate xanthan gum isomalt hexanediol caprylyl chlorphenesin phenoxyethanol tocopherol benzoate phenylpropanol glyceryl symphytum officinale callus culture
the moisturizing soft cream,moisturizer,la mer,"[combination, dry, normal, oily, sensitive, alltypes]",3.8,"[algae, seaweed, extract, cyclopentasiloxane, petrolatum, glyceryl, distearate, phenyl, trimethicone, butylene, glycol, hydrogenated, vegetable, oil, cholesterol, butyrospermum, parkii, shea, butter, steareth,

In [700]:
dataframe.shape

(1472, 6)

In [701]:
len(dataframe["bow"].iloc(0)[0].split(" "))

80

In [702]:
count = CountVectorizer()
count_matrix = count.fit_transform(dataframe["bow"])

In [703]:
count_matrix.shape

(1472, 4360)

In [704]:
dataframe.index

Index(['creme de la mer', 'facial treatment essence',
       'protini polypeptide cream', 'the moisturizing soft cream',
       'your skin but better cc cream with spf 50', 'the water cream',
       'lala retro whipped cream', 'virgin marula luxury facial oil',
       'ultra facial cream',
       'little miss miracle limitededition creme de la mer',
       ...
       'summer duo', 'passionflower  argan dry oil broad spectrum spf 50',
       'antiaging armour super smart skinperfecting beauty fluid spf 50',
       'naked skin bronzing beauty balm broad spectrum spf 20',
       'daily deflector moisturizer broad spectrum spf 50',
       'yoghurt nourishing fluid veil face sunscreen broad spectrum spf 30',
       'daily deflector waterlight broad spectrum spf 50 pa antiaging sunscreen',
       'self tan dry oil spf 50', 'pro light self tan bronzing mist',
       'dermaprotect daily defense broad spectrum spf 50'],
      dtype='object', name='Clean Name', length=1472)

In [705]:
list(count.vocabulary_.items())

[('moisturizer', 2641),
 ('combination', 1043),
 ('dry', 1396),
 ('normal', 2770),
 ('oily', 2846),
 ('sensitive', 3648),
 ('alltypes', 266),
 ('algae', 253),
 ('seaweed', 3630),
 ('extract', 1567),
 ('mineral', 2622),
 ('oil', 2841),
 ('petrolatum', 3080),
 ('glycerin', 1803),
 ('isohexadecane', 2148),
 ('microcrystalline', 2613),
 ('wax', 4289),
 ('lanolin', 2265),
 ('alcohol', 247),
 ('citrus', 980),
 ('aurantifolia', 451),
 ('lime', 2345),
 ('sesamum', 3662),
 ('indicum', 2091),
 ('sesame', 3661),
 ('seed', 3636),
 ('eucalyptus', 1530),
 ('globulus', 1773),
 ('leaf', 2296),
 ('powder', 3300),
 ('medicago', 2533),
 ('sativa', 3600),
 ('alfalfa', 252),
 ('helianthus', 1911),
 ('annuus', 342),
 ('sunflower', 3882),
 ('seedcake', 3638),
 ('prunus', 3366),
 ('amygdalus', 314),
 ('dulcis', 1401),
 ('sweet', 3901),
 ('almond', 268),
 ('meal', 2530),
 ('sodium', 3731),
 ('gluconate', 1781),
 ('copper', 1082),
 ('calcium', 728),
 ('magnesium', 2449),
 ('zinc', 4352),
 ('sulfate', 3872),
 ('

In [706]:
# cosine_similarity?

In [707]:
len(cosine_similarity(count_matrix, count_matrix)[1])

1472

In [708]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.27850333, 0.17611749, ..., 0.22417942, 0.19944037,
        0.22645541],
       [0.27850333, 1.        , 0.27824334, ..., 0.28460499, 0.32225169,
        0.4472136 ],
       [0.17611749, 0.27824334, 1.        , ..., 0.20530596, 0.18264968,
        0.13826023],
       ...,
       [0.22417942, 0.28460499, 0.20530596, ..., 1.        , 0.31703757,
        0.37712362],
       [0.19944037, 0.32225169, 0.18264968, ..., 0.31703757, 1.        ,
        0.42700841],
       [0.22645541, 0.4472136 , 0.13826023, ..., 0.37712362, 0.42700841,
        1.        ]])

In [709]:
indices = pd.Series(dataframe.index)

indices

0                                                               creme de la mer
1                                                      facial treatment essence
2                                                     protini polypeptide cream
3                                                   the moisturizing soft cream
4                                     your skin but better cc cream with spf 50
                                         ...                                   
1467         yoghurt nourishing fluid veil face sunscreen broad spectrum spf 30
1468    daily deflector waterlight broad spectrum spf 50 pa antiaging sunscreen
1469                                                    self tan dry oil spf 50
1470                                           pro light self tan bronzing mist
1471                           dermaprotect daily defense broad spectrum spf 50
Name: Clean Name, Length: 1472, dtype: object

In [710]:
indices[indices == "lala retro whipped cream"].index[0]

np.int64(6)

In [711]:
pd.Series(cosine_sim[1]).sort_values(ascending=False)[1:5+1]

1      1.000000
116    1.000000
828    0.667424
322    0.661724
870    0.588172
dtype: float64

In [712]:
dataframe.iloc[116]

Label                                                                                                                                                                           moisturizer
Brand                                                                                                                                                                                 sk-ii
Skin Types                                                                                                                            [combination, dry, normal, oily, sensitive, alltypes]
Rank                                                                                                                                                                                    0.0
keywords                                                   [galactomyces, ferment, filtrate, butylene, glycol, pentylene, water, sodium, benzoate, methylparaben, sorbic, acid, pitera, ™.]
bow           moisturizer combination dry normal oily sensit

In [713]:
dataframe.iloc[1]

Label                                                                                                                                                                        moisturizer
Brand                                                                                                                                                                              sk-ii
Skin Types                                                                                                                         [combination, dry, normal, oily, sensitive, alltypes]
Rank                                                                                                                                                                                 4.1
keywords                                                    [galactomyces, ferment, filtrate, pitera, butylene, glycol, pentylene, water, sodium, benzoate, methylparaben, sorbic, acid]
bow           moisturizer combination dry normal oily sensitive alltypes ga

In [714]:
def recommend_cosmetic(name, n = 5):
    recommended_list = []
    idx = indices[indices == name].index[0]

    socre_series = pd.Series(cosine_sim[idx]).sort_values(ascending=False)

    top_n_indexes = list(socre_series.iloc[1:n+1].index)
    print(top_n_indexes)

    for i in top_n_indexes:
        recommended_list.append(list(dataframe.index)[i])
    return dataframe.loc[recommended_list]

In [715]:
name = "baggage claim rose gold eye masks"
recommend_cosmetic(name)

[1116, 1108, 1150, 1274, 866]


,Label,Brand,Skin Types,Rank,keywords,bow
Clean Name,,,,,,
baggage claim gold eye masks,eye cream,wander beauty,"[combination, dry, normal, oily, sensitive, alltypes]",4.3,"[water, glycerin, butylene, glycol, 1, 2, hexanediol, aloe, barbadensis, leaf, juice, xanthan, gum, polyglyceryl, 4, caprate, panthenol, hydroxyethylcellulose, portulaca, oleracea, extract, calendula, officinalis, flower, myrciaria, dubia, fruit, butyrospermum, parkii, shea, butter, ethylhexylglycerin, disodium, edta, scutellaria, baicalensis, root, glyceryl, caprylate, sodium, hyaluronate, tocopheryl, acetate, simmondsia, chinensis, jojoba, seed, oil, lavandula, angustifolia, lavender, lecithin, cyclopentasiloxane, hydrogenated, natto, glycine, serine, glutamic, acid, beta, glucan, aspartic, leucine, alanine, lysine, arginine, tyrosine, phenylalanine, threonine, proline, valine, isoleucine, histidine, methionine, cysteine, acetyl, hexapeptide, 8, copper, tripeptide]",eye cream combination dry normal oily sensitive alltypes water glycerin butylene glycol 1 2 hexanediol aloe barbadensis leaf juice xanthan gum polyglyceryl 4 caprate panthenol hydroxyethylcellulose portulaca oleracea extract calendula officinalis flower myrciaria dubia fruit butyrospermum parkii shea butter ethylhexylglycerin disodium edta scutellaria baicalensis root glyceryl caprylate sodium hyaluronate tocopheryl acetate simmondsia chinensis jojoba seed oil lavandula angustifolia lavender lecithin cyclopentasiloxane hydrogenated natto glycine serine glutamic acid beta glucan aspartic leucine alanine lysine arginine tyrosine phenylalanine threonine proline valine isoleucine histidine methionine cysteine acetyl hexapeptide 8 copper tripeptide
bye bye under eye eye cream smooths brightens depuffs,eye cream,it cosmetics,"[combination, dry, normal, oily, sensitive, alltypes]",3.7,"[water, cyclopentasiloxane, butylene, glycol, butyrospermum, parkii, shea, butter, dimethicone, niacinamide, cetearyl, alcohol, peg, 40, stearate, palmitic, acid, stearic, polysorbate, 60, macadamia, ternifolia, seed, oil, cetyl, ethylhexanoate, glyceryl, glycerin, acrylates, c10, 30, alkyl, acrylate, crosspolymer, carbomer, sodium, hyaluronate, acetyl, hexapeptide, 8, copper, gluconate, collagen, tea, cocoyl, glutamate, palmitoyl, pentapeptide, 4, tripeptide, 1, biotin, caffeine, magnesium, aspartate, zinc, ascorbyl, phosphate, lauroyl, lactylate, ceramide, np, ap, phytosphingosine, cholesterol, xanthan, gum, eop, retinol, tocopheryl, acetate, ascorbic, aloe, barbadensis, leaf, extract, anthemis, nobilis, flower, curcuma, longa, turmeric, root, squalane, spirulina, platensis, algae, cucumis, sativus, cucumber, fruit, colloidal, oatmeal, propylene, glycyrrhiza, glabra, licorice, camellia, sinensis, lecithin, ...]",eye cream combination dry normal oily sensitive alltypes water cyclopentasiloxane butylene glycol butyrospermum parkii shea butter dimethicone niacinamide cetearyl alcohol peg 40 stearate palmitic acid stearic polysorbate 60 macadamia ternifolia seed oil cetyl ethylhexanoate glyceryl glycerin acrylates c10 30 alkyl acrylate crosspolymer carbomer sodium hyaluronate acetyl hexapeptide 8 copper gluconate collagen tea cocoyl glutamate palmitoyl pentapeptide 4 tripeptide 1 biotin caffeine magnesium aspartate zinc ascorbyl phosphate lauroyl lactylate ceramide np ap phytosphingosine cholesterol xanthan gum eop retinol tocopheryl acetate ascorbic aloe barbadensis leaf extract anthemis nobilis flower curcuma longa turmeric root squalane spirulina platensis algae cucumis sativus cucumber fruit colloidal oatmeal propylene glycyrrhiza glabra licorice camellia sinensis lecithin caprylic capric triglyceride creatine simmondsia chinensis jojoba vaccinium angustifolium blueberry sea panthenol polyglutamic methionine lysine leucine histidine isoleucine arginine disodium edta polyacrylate 13 polyisobutene 20 sorbitan isostearate tromethamine benzophenone 5 cholesteryl nonanoate chloride phenylalanine threonine valine saccharid

In [716]:
dataframe.loc[name]

Label                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         eye cream
Brand                                                                                                                                                                                                           